In [3]:
import pandas as pd
import numpy as np
import os
import scipy.stats as stats
from utils import load_data, val_loglhood, loglhood

# Gibbs Sampling

In [ ]:
def gibbs_sampling(iters, data_path, K, p, q, debug=False):
    """
    data_path: path where data is saved.
    K: number of plants (n_plants in load_data function).
    p: time interval to consider.
    q: sample distribution for parameters.
    """
    if debug:
        print('Loading data...')
    Y0, X = load_data(data_path, K, p, resample_rule='20H')
    if debug:
        print('Y0 shape: {}'.format(Y0.shape))
        print('X shape: {}'.format(X.shape))
    theta = init_parameters(K, p, q, Y0, X, debug)
    if debug:
        print('Parameters intialized!')
    samples = []
    for i in range(iters):
        if debug:
            print('Iteration {}'.format(i))
        for j in theta.shape[0]:
            while True:
                theta_j = q.rvs()
                lk = val_loglhood(theta, Y0, X, True)
                if lk != -np.inf:
                    break
        samples.append(theta)
    return np.array(samples)
        
    
def init_parameters(K, p, q, Y0, X, debug=False):
    if debug:
        print('Initializing parameters...')
    while True:
        theta = np.zeros(K ** 2 * (p + 1))
        for i in range(theta.shape[0]):
            theta[i] = q.rvs()

        # Force CovU to be positive semidefinite.
        covu = np.reshape(theta[-K**2:], (K, K)).T
        covu = np.dot(covu.T, covu)
        theta[-K**2:] = np.reshape(covu, K**2)
        
        lk = val_loglhood(theta, Y0, X, True)
        print('LK = {}'.format(lk))
        if lk != -np.inf:
            return theta
        

In [ ]:
a = np.arange(20)
print(a)
b = a[-3 ** 2:]
print(b)
c = np.reshape(b, (3, 3)).T
print(c)
print(np.dot(c.T, c))
d = np.reshape(b, 3 ** 2)
print(d)

In [5]:
DATA_PATH = '/home/chrisams/Documents/datasets/data_TAIM/processed/'
q = stats.norm(loc=0, scale=1)
K = 2
p = 1
iters = 5
debug = True

In [ ]:
gibbs_sampling(iters, DATA_PATH, K, p, q, debug)

In [6]:
covu = np.array([[1.37384213, -2.91849886], [-2.91849886, 6.22740396]])
A = np.array([[-1.11480087, -1.14222902], [0.36701172, 0.57859253]])
Y0, X = load_data(DATA_PATH, K, p, resample_rule='4H')

(2, 16611)


In [7]:
loglhood(covu, Y0, A, X)

nan

In [8]:
trace_mat = np.transpose(Y0 - A@X) @ np.linalg.inv(covu) @ (Y0 - A@X)
print(trace_mat)
print(trace_mat.shape)

[[  166.02291698    36.75488817   472.44638924 ...  -241.18389599
   -492.48821635  -409.88683039]
 [   36.75488817     8.47677686   104.95995004 ...   -53.66666186
   -109.18224222   -90.90043592]
 [  472.44638924   104.95995004  1344.82421043 ...  -686.62434065
  -1401.62437611 -1166.57336458]
 ...
 [ -241.18389599   -53.66666186  -686.62434065 ...   350.58954201
    715.56738403   595.57504406]
 [ -492.48821635  -109.18224222 -1401.62437611 ...   715.56738403
   1460.9795301   1215.95404158]
 [ -409.88683039   -90.90043592 -1166.57336458 ...   595.57504406
   1215.95404158  1012.02529197]]
(16610, 16610)


In [ ]:
out = -(Y0.shape[1]/2)*np.log(np.linalg.det(covu)) -1/2*np.trace(trace_mat) 
print(out)

In [ ]:
np.log(np.linalg.det(covu))

In [9]:
sum(np.diag(trace_mat))

nan

In [28]:
def how_many_nans(L):
    L = L.reshape(L.shape[0] * L.shape[1])
    c = 0
    for i in L:
        if np.isnan(i):
            c+=1
    print(c)

In [29]:
print(c)

1330


In [30]:
np.diag(trace_mat).shape

(16610,)

In [32]:
how_many_nans(X)

1325
